In [ ]:
#数据质量
# 一 有效性：审核有效性是关于确定对个别字段施加什么约束并检查以确保字段值遵守这些约束条件
# 跨字段限制：当一个数据项拥有多个字段时，它们必须在某种程度上具有一致性
#准确率
#完整性
#一致性
#统一性

#使用蓝图的示例
#!/usr/bin/env python
          # -*- coding: utf-8 -*-
          import xml.etree.cElementTree as ET
          from collections import defaultdict
          import re

          osm_file = open("chicago.osm", "r")

          street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
          street_types = defaultdict(int)

          def audit_street_type(street_types, street_name):
              m = street_type_re.search(street_name)
              if m:
                  street_type = m.group()

                  street_types[street_type] += 1

          def print_sorted_dict(d):
              keys = d.keys()
              keys = sorted(keys, key=lambda s: s.lower())
              for k in keys:
                  v = d[k]
                  print "%s: %d" % (k, v) 

          def is_street_name(elem):
              return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

          def audit():
              for event, elem in ET.iterparse(osm_file):
                  if is_street_name(elem):
                      audit_street_type(street_types, elem.attrib['v'])    
              print_sorted_dict(street_types)    

          if __name__ == '__main__':
              audit()

In [1]:
#练习: 修正有效性
# Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
# The following things should be done:
# - check if the field "productionStartYear" contains a year
# - check if the year is in range 1886-2014
# - convert the value of the field to be just a year (not full datetime)
# - the rest of the fields and values should stay the same
# - if the value of the field is a valid year in the range as described above,
#   write that line to the output_good file
# - if the value of the field is not a valid year as described above, 
#   write that line to the output_bad file
# - discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
# - you should use the provided way of reading and writing data (DictReader and DictWriter)
#   They will take care of dealing with the header.
# 你的任务是检查 DBPedia 自动数据文件的“productionStartYear”并获取有效的值。应该完成以下任务：
# -    检查字段“productionStartYear”是否包含年份
# -    检查该年份是否在 1886 至 2014 范围内
# -    将字段值转换为年份（而不是整个日期时间）
# -    字段的其他部分和值应该保持不变
# -    如果字段的值是如上所述范围内的有效年份，则将该行写入 output_good 文件中
# -    如果字段的值不是如上所述的有效年份，则将该行写入 output_bad 文件中
# -    你应该采用提供的数据读取和写入方式（DictReader 和 DictWriter），它们将会对标题进行处理。
# 
# You can write helper functions for checking the data and writing the files, but we will call only the 
# 'process_file' with 3 arguments (inputfile, output_good, output_bad).
# 你可以编写辅助函数来检查数据并编写文件，
# 但是我们将仅调用包含三个参数（inputfile、output_good、output_bad）的“process_file”文件。

import csv
import pprint
from datetime import datetime
import re

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'
good = []
bad = []

def process_file(input_file, output_good, output_bad):

    # with open(input_file, "r") as f:
    #     reader = csv.DictReader(f)
    #     header = reader.fieldnames
    #     print header
    
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        # good.append(header)
        # bad.append(header)
        p = re.compile('\d{4}', re.IGNORECASE)
        for row in reader:
            if row['URI'].find("dbpedia.org") < 0:
                continue
            m =re.search('\d{4}',row['productionStartYear'])
            if m:
                year = int(m.group())
                if year > 1886 and year <  2014:
                     row['productionStartYear'] = year
                     good.append(row)
                else:
                    row['productionStartYear'] = year
                    bad.append(row)
                    
            else:
                bad.append(row)
        

        #COMPLETE THIS FUNCTION



    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    
    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in good:
            writer.writerow(row)
            
            
    with open(output_bad, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in bad:
            writer.writerow(row)
            


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)
    


    
        # print datetime.strptime(row['productionStartYear'],"%Y%d%m")
        
        
    
test()

# ANSWER:
# def process_file(input_file, output_good, output_bad):
#     # store data into lists for output
#     data_good = []
#     data_bad = []
#     with open(input_file, "r") as f:
#         reader = csv.DictReader(f)
#         header = reader.fieldnames
#         for row in reader:
#             # validate URI value
#             if row['URI'].find("dbpedia.org") < 0:
#                 continue
# 
#             ps_year = row['productionStartYear'][:4]
#             try: # use try/except to filter valid items
#                 ps_year = int(ps_year)
#                 row['productionStartYear'] = ps_year
#                 if (ps_year >= 1886) and (ps_year <= 2014):
#                     data_good.append(row)
#                 else:
#                     data_bad.append(row)
#             except ValueError: # non-numeric strings caught by exception
#                 if ps_year == 'NULL':
#                     data_bad.append(row)
# 
#     # Write processed data to output files
#     with open(output_good, "w") as good:
#         writer = csv.DictWriter(good, delimiter=",", fieldnames= header)
#         writer.writeheader()
#         for row in data_good:
#             writer.writerow(row)
# 
#     with open(output_bad, "w") as bad:
#         writer = csv.DictWriter(bad, delimiter=",", fieldnames= header)
#         writer.writeheader()
#         for row in data_bad:
#             writer.writerow(row)

In [2]:
#练习: 审查数据质量
# In this problem set you work with cities infobox data, audit it, come up with a
# cleaning idea and then clean it up. In the first exercise we want you to audit
# the datatypes that can be found in some particular fields in the dataset.
# The possible types of values can be:
# - NoneType if the value is a string "NULL" or an empty string ""
# - list, if the value starts with "{"
# - int, if the value can be cast to int
# - float, if the value can be cast to float, but CANNOT be cast to int.
#    For example, '3.23e+07' should be considered a float because it can be cast
#    as float but int('3.23e+07') will throw a ValueError
# - 'str', for all other values
# 在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。在第一道练习中，请审核
# 数据集中某些特定字段中的数据类型。
# 值类型可以是：
# -    NoneType，如果值是字符串“NULL”或空字符串“”
# -    列表，如果值以“{”开头
# -    整型，如果值可以转型为整型
# -    浮点型，如果值可以转型为浮点型，但是无法转型为整型。
# 例如，“3.23e+07”应该被当做浮点型，因为可以转型为浮点型，但是int('3.23e+07') 将抛出 ValueError
# -    “str”，表示其他所有值
# 
# The audit_file function should return a dictionary containing fieldnames and a 
# SET of the types that can be found in the field. e.g.
# {"field1": set([type(float()), type(int()), type(str())]),
#  "field2": set([type(str())]),
#   ....
# }
# The type() function returns a type object describing the argument given to the 
# function. You can also use examples of objects to create type objects, e.g.
# type(1.1) for a float: see the test function below for examples.
# audit_file 函数应该返回一个字典，其中包含字段名称和可以在该字段中找到的类型集。例如
# {"field1": set([type(float()), type(int()), type(str())]),
#  "field2": set([type(str())]),
#   ....
# }
# type() 函数返回的是类型对象，描述了提供给该函数的参数。你还可以使用对象示例创建类型对象，例如 type(1.1) 
# 表示浮点型：具体示例请参阅下面的测试函数。
# 
# Note that the first three rows (after the header row) in the cities.csv file
# are not actual data points. The contents of these rows should note be included
# when processing data types. Be sure to include functionality in your code to
# skip over or detect these rows.
# 注意，cities.csv 文件的前三行（标题行之后）不是实际的数据点。在处理数据类型时，不应该包含这些行的内容。
# 确保在代码中包含相关功能，以便跳过或检测出这些行。

import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def audit_file(filename, fields):
    fieldtypes = {}

    # YOUR CODE HERE


    return fieldtypes

with open(CITIES,'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        print row
# def test():
#     fieldtypes = audit_file(CITIES, FIELDS)
# 
#     pprint.pprint(fieldtypes)
# 
#     assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
#     assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
#     
# if __name__ == "__main__":
#     test()

{'code': 'http://dbpedia.org/ontology/code', 'populationRural': 'http://dbpedia.org/ontology/populationRural', 'state_label': 'http://dbpedia.org/ontology/state', 'twinCity_label': 'http://dbpedia.org/ontology/twinCity', 'mayor_label': 'http://dbpedia.org/ontology/mayor', 'mayor': 'http://dbpedia.org/ontology/mayor', 'governingBody_label': 'http://dbpedia.org/ontology/governingBody', 'areaLand': 'http://dbpedia.org/ontology/areaLand', 'rdf-schema#label': 'http://www.w3.org/2000/01/rdf-schema#label', 'foundingPerson_label': 'http://dbpedia.org/ontology/foundingPerson', 'division_label': 'http://dbpedia.org/ontology/division', 'location': 'http://dbpedia.org/ontology/location', 'foundingYear': 'http://dbpedia.org/ontology/foundingYear', 'homepage': 'http://xmlns.com/foaf/0.1/homepage', 'division': 'http://dbpedia.org/ontology/division', 'elevation': 'http://dbpedia.org/ontology/elevation', 'government': 'http://dbpedia.org/ontology/government', 'URI': 'URI', 'isPartOf_label': 'http://dbp

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
